In [1]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import torch.nn as nn
from torch.utils.data import  DataLoader, Dataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm
import scipy.io
import os

/Users/mclinwong/miniforge3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Seed is important, especially training the CNN+ReLU Net
#Otherwise, the results will be very different
myseed = 8974  # set a random seed for reproducibility
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(myseed)
torch.manual_seed(myseed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(myseed)

In [3]:
#Train&Test Data library
matlib = '/Users/mclinwong/GitHub/CodesReproduction/DCN-DOA/Data/matlib'
#Path to save figures
figure_savepath = '/Users/mclinwong/GitHub/CodesReproduction/DCN-DOA/ReproducedCodes/Figures/'
#Path to save the model
pthpath = '/Users/mclinwong/GitHub/CodesReproduction/DCN-DOA/Data/pth/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "mps")
num_epoch = 600
batch_size = 64
learning_rate = 0.001
# load model
# autoencoder = torch.load('/Users/mclinwong/GitHub/CodesReproduction/DCN-DOA/ReproducedCodes/matlib/11_22/autoencoder.h5')

In [4]:
datapath = os.path.join(matlib, 'data2_trainlow.mat')
read_data = scipy.io.loadmat(datapath)
S_est = read_data['S_est']
S_abs = read_data['S_abs']
S_label = read_data['S_label']
R_est = read_data['R_est']
S_label1 = np.expand_dims(S_label, 2)
[Sample, L, dim] = np.shape(S_est)
S_est = S_est.transpose(0, 2, 1)
S_label1 = S_label1.transpose(0, 2, 1)

print(f'S_est.shape: {S_est.shape}')
print(f'S_abs.shape: {S_abs.shape}')
print(f'S_label.shape: {S_label.shape}')
print(f'S_label1.shape: {S_label1.shape}')
print(f'Sample: {Sample}, L: {L}, dim: {dim}')

S_est.shape: (19800, 2, 120)
S_abs.shape: (19800, 240)
S_label.shape: (19800, 120)
S_label1.shape: (19800, 1, 120)
Sample: 19800, L: 120, dim: 2


In [5]:
class MakeDataset(Dataset):
    def __init__(self, data, label):
        self.data = data
        self.label = label
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        label = self.label[idx]
        data = self.data[idx]
        return data, label

In [6]:
S_est_train, S_est_test, S_label1_train, S_label1_test = train_test_split(S_est, S_label1, test_size=0.2)
S_abs_train, S_abs_test, S_label_train, S_label_test = train_test_split(S_abs, S_label, test_size=0.2)
print(f'S_est_train.shape: {S_est_train.shape}, S_est_test.shape: {S_est_test.shape}')
print(f'S_abs_train.shape: {S_abs_train.shape}, S_abs_test.shape: {S_abs_test.shape}')
print(f'S_label1_train.shape: {S_label1_train.shape}, S_label1_test.shape: {S_label1_test.shape}')

train_set = MakeDataset(S_est_train, S_label1_train)
train_set_fcn = MakeDataset(S_abs_train, S_label_train)
valid_set = MakeDataset(S_est_test, S_label1_test)
valid_set_fcn = MakeDataset(S_abs_test, S_label_test)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False)
train_loader_fcn = DataLoader(train_set_fcn, batch_size=batch_size, shuffle=True)
valid_loader_fcn = DataLoader(valid_set_fcn, batch_size=batch_size, shuffle=False)

S_est_train.shape: (15840, 2, 120), S_est_test.shape: (3960, 2, 120)
S_abs_train.shape: (15840, 240), S_abs_test.shape: (3960, 240)
S_label1_train.shape: (15840, 1, 120), S_label1_test.shape: (3960, 1, 120)


# Define Nets

In [7]:
# Shape of Conv1D: (batch_size, channels, seq_len)
# length_out = (length_in - kernel_size + 2 * padding) / stride + 1
class CNN(nn.Module):
    def __init__(self,activ):
        super().__init__()
        self.conv1 = nn.Sequential(nn.Conv1d(2,12,kernel_size=25, padding=12), nn.BatchNorm1d(12), activ())
        self.conv2 = nn.Sequential(nn.Conv1d(12,6,kernel_size=15, padding=7),  nn.BatchNorm1d(6), activ())
        self.conv3 = nn.Sequential(nn.Conv1d(6,3,kernel_size=5, padding=2), nn.BatchNorm1d(3), activ())
        self.conv4 = nn.Sequential(nn.Conv1d(3,1,kernel_size=3, padding=1),nn.BatchNorm1d(1), activ())
    
    def forward(self, x):  
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.conv3(out)
        out = self.conv4(out)
       
        return out

In [8]:
#L=120, 
#in_num = 2*L = 240
#out1_num = int(2*L/3) = 80
#out2_num = int(4*L/9) = 53
#out3_num = int(2*L/3) = 80
#out4_num = L = 120
class DNN(nn.Module):
    def __init__(self, activa):
        super(DNN, self).__init__()
        self.fc1 = nn.Sequential(nn.Linear(2*L, int(2*L/3)), nn.BatchNorm1d(80), activa())
        self.fc2 = nn.Sequential(nn.Linear(int(2*L/3), int(4*L/9)), nn.BatchNorm1d(53), activa())
        self.fc3 = nn.Sequential(nn.Linear(int(4*L/9), int(2*L/3)), nn.BatchNorm1d(80), activa())
        self.fc4 = nn.Sequential(nn.Linear(int(2*L/3), L), nn.BatchNorm1d(120), activa())
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        return out

# TRAIN Function

In [9]:
def train(model, train_loader, valid_loader, epoch, name):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.MSELoss()
    train_loss_list = []
    valid_loss_list = []
    
    for epoch in range(num_epoch):
        model.train()
        train_loss = []
        
        for x, y in tqdm(train_loader):
            optimizer.zero_grad()
            x = x.to('cpu').float()
            x = x.to(device)
            y = y.to('cpu').float()
            y = y.to(device)
            output = model(x)
            loss = criterion(output, y)
            loss.backward()
            optimizer.step()
            train_loss.append(loss.item())
            
        train_loss = np.mean(train_loss)
        train_loss_list.append(train_loss)
        print('Epoch: {}, Train Loss: {:.4f}'.format(epoch, train_loss))
    
        model.eval()
        valid_loss = []
        
        for x, y in tqdm(valid_loader):
            x = x.to('cpu').float()
            x = x.to(device)
            y = y.to('cpu').float()
            y = y.to(device)
            with torch.no_grad():
                output = model(x)
            loss = criterion(output, y)
            valid_loss.append(loss.item())
            
        valid_loss = np.mean(valid_loss)
        valid_loss_list.append(valid_loss)
        print('Epoch: {}, Valid Loss: {:.4f}'.format(epoch, valid_loss))
    
    #save loss as csv
    id = np.arange(0, num_epoch)
    datafarme = pd.DataFrame({'id':id ,'train_loss':train_loss_list, 'valid_loss':valid_loss_list})
    datafarme.to_csv(figure_savepath + str(name) +'loss.csv', index=False, sep=',')
    return train_loss_list, valid_loss_list

# Train & Save models

In [10]:


CNN_ReLU = CNN(nn.ReLU).to(device)
CNN_Tanh = CNN(nn.Tanh).to(device)
CNN_Sigmoid = CNN(nn.Sigmoid).to(device)

DNN_ReLU = DNN(nn.ReLU).to(device)
DNN_Tanh = DNN(nn.Tanh).to(device)
DNN_Sigmoid = DNN(nn.Sigmoid).to(device)

tll_cnn_relu, vll_cnn_relu = train(
    CNN_ReLU, train_loader, valid_loader, num_epoch, 'CNN_ReLU')
torch.save(CNN_ReLU, pthpath + 'cnnrelu.pth')

tll_cnn_tanh, vll_cnn_tanh = train(
    CNN_Tanh, train_loader, valid_loader, num_epoch, 'CNN_Tanh')
torch.save(CNN_Tanh, pthpath + 'cnntanh.pth')

tll_cnn_sigmoid, vll_cnn_sigmoid = train(
    CNN_Sigmoid, train_loader, valid_loader, num_epoch, 'CNN_Sigmoid')
torch.save(CNN_Sigmoid, pthpath + 'cnnsigmoid.pth')
 
tll_dnn_relu, vll_dnn_relu = train(
    DNN_ReLU, train_loader_fcn, valid_loader_fcn, num_epoch, 'DNN_ReLU')
torch.save(DNN_ReLU, pthpath + 'dnnrelu.pth')

tll_dnn_tanh, vll_dnn_tanh = train(
    DNN_Tanh, train_loader_fcn, valid_loader_fcn, num_epoch, 'DNN_Tanh')
torch.save(DNN_Tanh, pthpath + 'dnntanh.pth')

tll_dnn_sigmoid, vll_dnn_sigmoid = train(
    DNN_Sigmoid, train_loader_fcn, valid_loader_fcn, num_epoch, 'DNN_Sigmoid')
torch.save(DNN_Sigmoid, pthpath + 'dnnsigmoid.pth')

100%|██████████| 248/248 [00:04<00:00, 54.25it/s]


Epoch: 0, Train Loss: 0.0596


100%|██████████| 62/62 [00:00<00:00, 216.17it/s]


Epoch: 0, Valid Loss: 0.0164


100%|██████████| 248/248 [00:03<00:00, 63.49it/s]


...

100%|██████████| 248/248 [00:03<00:00, 65.73it/s]


Epoch: 11, Train Loss: 0.0114


100%|██████████| 62/62 [00:00<00:00, 248.67it/s]


Epoch: 11, Valid Loss: 0.0117


 20%|██        | 50/248 [00:00<00:02, 68.08it/s]

# Save ,Plot

In [ ]:
tll_cnn_relu_mult1000 = [i*1000 for i in tll_cnn_relu]
vll_cnn_relu_mult1000 = [i*1000 for i in vll_cnn_relu]
tll_cnn_tanh_mult1000 = [i*1000 for i in tll_cnn_tanh]
vll_cnn_tanh_mult1000 = [i*1000 for i in vll_cnn_tanh]
tll_dnn_relu_mult1000 = [i*1000 for i in tll_dnn_relu]
vll_dnn_relu_mult1000 = [i*1000 for i in vll_dnn_relu]
tll_dnn_tanh_mult1000 = [i*1000 for i in tll_dnn_tanh]
vll_dnn_tanh_mult1000 = [i*1000 for i in vll_dnn_tanh]

# save as csv
import pandas as pd
epcs = np.arange(0, num_epoch)
df = pd.DataFrame({'epcho':epcs, 'tll_cnn_relu_mult1000':tll_cnn_relu_mult1000, 'vll_cnn_relu_mult1000':vll_cnn_relu_mult1000, 'tll_cnn_tanh_mult1000':tll_cnn_tanh_mult1000, 'vll_cnn_tanh_mult1000':vll_cnn_tanh_mult1000, 'tll_dnn_relu_mult1000':tll_dnn_relu_mult1000, 'vll_dnn_relu_mult1000':vll_dnn_relu_mult1000, 'tll_dnn_tanh_mult1000':tll_dnn_tanh_mult1000, 'vll_dnn_tanh_mult1000':vll_dnn_tanh_mult1000})
df.to_csv(figure_savepath + 'loss.csv', index=False)

In [ ]:
with plt.style.context(['science']):
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    plt.ylim(0, 20)
    plt.plot(tll_cnn_relu_mult1000, label='DCN relu')
    plt.plot(tll_cnn_tanh_mult1000, label='DCN Tanh')
    plt.plot(tll_dnn_relu_mult1000, label='DNN relu')
    plt.plot(tll_dnn_tanh_mult1000, label='DNN Tanh')
    plt.xlabel('Epoch')
    plt.ylabel('Train MSE ($ *10^{-3}$)')
    plt.legend()
    #save figure
    plt.savefig(figure_savepath + 'Train Loss_compare.pdf')
    plt.show()

In [ ]:
with plt.style.context(['science']):
    plt.rcParams['figure.dpi'] = 300
    plt.rcParams['savefig.dpi'] = 300
    plt.ylim(0, 20)
    plt.plot(valid_loss_list_cnn_ReLu_mult1000, label='DCN ReLU')
    plt.plot(valid_loss_list_cnn_tanh_mult1000, label='DCN Tanh')
    plt.plot(valid_loss_list_dnn_ReLu_mult1000, label='DNN ReLu')
    plt.plot(valid_loss_list_dnn_Tanh_mult1000, label='DNN Tanh')
    plt.xlabel('Epoch')
    plt.ylabel('Valid MSE ($ *10^{-3}$)')
    plt.legend()
    #save figure
    plt.savefig(figure_savepath + 'Valid Loss_compare.pdf')
    plt.show()